# Diagonalización de matrices simétricas por semejanza ortogonal

Para ilustrar cómo diagonalizar matrices simétricas por semejanza ortogonal, vamos a usar el Ejemplo 9 (Capítulo IV) de [L. Merino, E. Santos [Álgebra Lineal con Métodos Elementales](https://www.amazon.es/%C3%81lgebra-lineal-m%C3%A9todos-elementales-GONZALEZ/dp/8497324811)]. 

## Ejemplo

*Encuentra una matriz ortogonal $P$ tal que $P^tAP$ sea diagonal, con*
$$
A=\begin{pmatrix}3 & 1 & 1\\
1 & 3 & 1\\
1 & 1 & 3
\end{pmatrix}.
$$


In [1]:
A<-rbind(c(3,1,1),c(1,3,1),c(1,1,3))
A

3,1,1
1,3,1
1,1,3


### Con `pracma`

Vamos a hacer uso de `nullspace` que calcula una base ortonormal del núcleo de una aplicación lineal.

In [2]:
library("pracma")

Empezamos calculando los valores propios.

In [3]:
eigen(A)$values

[1] 5 2 2

Luego calculamos una base ortonormal (tendrá sólo un vector) del subespacio propio asociado a 5.

In [4]:
A5<-A-5*diag(3)
V5<-nullspace(A5)
V5

0.5773503
0.5773503
0.5773503


Vemos que efectivamente el vector que nos proporciona tiene norma uno.

In [5]:
t(V5)%*%V5

1


Ahora calculamos una base ortonormal para el subespacio propio asociado al autovalor 2.

In [6]:
A2<-A-2*diag(3)
V2<-nullspace(A2)
V2

-0.5773503,-0.5773503
0.7886751,-0.2113249
-0.2113249,0.7886751


Y con esto ya tenemos la matriz de paso:

In [7]:
P<-cbind(V5,V2)
P

0.5773503,-0.5773503,-0.5773503
0.5773503,0.7886751,-0.2113249
0.5773503,-0.2113249,0.7886751


Comprobamos que efectivamente $P^tAP$ es diagonal.

In [8]:
zapsmall(t(P)%*%A%*%P)

5,0,0
0,2,0
0,0,2


### Con `caracas`

La ventaja de usar `caracas` es que manipula simbólicamente raíces y racionales, por lo que la salida es más precisa.

In [9]:
library("caracas")


Attaching package: ‘caracas’


The following objects are masked from ‘package:pracma’:

    inv, nullspace, pinv, rref, taylor


The following objects are masked from ‘package:base’:

    %*%, det, diag, diag<-




In [10]:
A<-as_sym(A)

In [11]:
eigenval(A)

[[1]]
[[1]]$eigval
[caracas]: 5

[[1]]$eigmult
[1] 1


[[2]]
[[2]]$eigval
[caracas]: 2

[[2]]$eigmult
[1] 2



La función `nullspace` de `caracas` no devuelve una base ortonormal.

In [12]:
A5<-A-5*as_sym(diag(3))
V5<-nullspace(A5)
V5

[[1]]
[caracas]: [1  1  1]ᵀ


In [13]:
v1<-V5[[1]]

Definamos una función que calcule la norma de un vector, y despues otra que normalice un vector.

In [14]:
norma<-function(v){
    return(sqrt(t(v)%*%v)[1])
}

In [15]:
normaliza<-function(v){
    return(cbind(v)/norma(v))
}

In [16]:
norma(v1)

[caracas]: √3

In [17]:
normaliza(v1)

[caracas]: ⎡√3  √3  √3⎤
           ⎢──  ──  ──⎥
           ⎣3   3   3 ⎦ᵀ

In [18]:
A2<-A-2*as_sym(diag(3))
V2<-nullspace(A2)
V2

[[1]]
[caracas]: [-1  1  0]ᵀ

[[2]]
[caracas]: [-1  0  1]ᵀ


Podemos usar `GramSchmidt` para encontrar una base ortogonal.

In [19]:
gV2<-GramSchmidt(cbind(V2[[1]],V2[[2]]))
gV2

[caracas]: ⎡-1  -1/2⎤
           ⎢        ⎥
           ⎢1   -1/2⎥
           ⎢        ⎥
           ⎣0    1  ⎦

Finalmente normalizamos todos los vectores, y construimos la matriz de paso.

In [20]:
u1<-normaliza(v1)
u2<-normaliza(gV2[,1])
u3<-normaliza(gV2[,2])
P<-cbind(u1,u2,u3)
P

[caracas]: ⎡√3  -√2   -√6 ⎤
           ⎢──  ────  ────⎥
           ⎢3    2     6  ⎥
           ⎢              ⎥
           ⎢√3   √2   -√6 ⎥
           ⎢──   ──   ────⎥
           ⎢3    2     6  ⎥
           ⎢              ⎥
           ⎢√3         √6 ⎥
           ⎢──   0     ── ⎥
           ⎣3          3  ⎦

Comprobamos que el resultado es correcto.

In [21]:
t(P)%*%A%*%P

[caracas]: ⎡5  0  0⎤
           ⎢       ⎥
           ⎢0  2  0⎥
           ⎢       ⎥
           ⎣0  0  2⎦

En vez de definir una función que normalice los vectores, podríamos haber usado `QRdecomposition`, que devuelve una descomposición QR, y por tanto una base ortonormal de los vectores que le pasemos por columnas.

In [22]:
QRdecomposition(cbind(V5[[1]]))

$Q
[caracas]: ⎡√3  √3  √3⎤
           ⎢──  ──  ──⎥
           ⎣3   3   3 ⎦ᵀ

$R
[caracas]: [√3]


In [23]:
QRdecomposition(cbind(V2[[1]],V2[[2]]))$Q

[caracas]: ⎡-√2   -√6 ⎤
           ⎢────  ────⎥
           ⎢ 2     6  ⎥
           ⎢          ⎥
           ⎢ √2   -√6 ⎥
           ⎢ ──   ────⎥
           ⎢ 2     6  ⎥
           ⎢          ⎥
           ⎢       √6 ⎥
           ⎢ 0     ── ⎥
           ⎣       3  ⎦